In [3]:
# Given values
noise_activity = 1  # This is the first value you mentioned
pr_passive = 0.7
trade_intensity = 0.4

# Calculation
inv = noise_activity * (1 - pr_passive) * trade_intensity / (1 - trade_intensity)

print(f"The calculated value is: {inv}")

The calculated value is: 0.20000000000000004
